In [1]:
import os
sep_local = os.path.sep
sep_local

In [2]:
%env TF_KERAS = 1

Using TensorFlow backend.


In [3]:
import sys
sys.path.append('..' + sep_local + '..' + sep_local + '..') # For Windows import

In [4]:
import os
os.chdir('..' + sep_local + '..' + sep_local + '..') # For Linux import


  DEBUG    | Looking for images in '0000'


  INFO     | 841 file found


  DEBUG    | Looking for images in '0001'


  INFO     | 1080 file found


  DEBUG    | Looking for images in '0002'


...

  INFO     | 994 file found


  DEBUG    | Looking for images in '0049'


  INFO     | 863 file found


  DEBUG    | Looking for images in '0050'


  INFO     | 648 file found


In [6]:
from training.generators.file_image_generator import create_image_lists, get_generators

  INFO     | Found 33971 training files


  INFO     | Found 12408 validation files


In [7]:
#IMG_DIR = 'D:\\datasets\\Ms_Pacman\\Ms_Pacman\\Test\\'
IMG_DIR = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux

In [8]:
#IMG_DIR = '..\..\..\\data\\atari_v1\\screens\\mspacman'
VAL_PCT = 25
VAL_FORMAT = 'png'

In [9]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

## downscalse the image into fifth

In [ ]:
scale=5
IMG_SIZE=(160//scale, 210//scale, 3)
BATCH_SIZE = 5
EPIS_LEN = 10
EPIS_SHIFT = 5

In [ ]:
#we created new  class_mode  episode_flat witch returns 
#da.from_array(np.reshape(batch_x, (-1,)+self.image_shape )), da.from_array(np.reshape(batch_gt, (-1,)+self.image_shape))

In [ ]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode='episode_flat', 
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
#changed output shapes of Dataset Generators 
#from (BATCH_SIZE, EPIS_LEN, ) + IMG_SIZE)
#to (BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE) 

In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE), 
                   tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE), 
                   tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE)
                  )
)


In [ ]:
dataset_name = 'atari_v1_Pacman'

In [ ]:
intermediate_dim = 10
inputs_shape= IMG_SIZE # image shape
LATENT_DIM = 10

In [11]:
inputs_shape

(6, 160, 210, 3)

In [12]:
tdConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(**kwds))

In [13]:
tdDeConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2DTranspose(**kwds))

In [14]:
tdDense = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(**kwds))

In [16]:
c = 3

In [17]:
enc_lays = [
    tdConv(filters=c, kernel_size=3, strides=(2, 2), activation='relu'),
    tdConv(filters=2*c, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(LATENT_DIM)
]


dec_lays = [
    tf.keras.layers.Dense(units=c**3, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(1, c, c, c)),
    tdDeConv(filters=2*c, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tdDeConv(filters=c, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tdDeConv(filters=1, kernel_size=3, strides=(1, 1), padding="SAME"),    
]

In [18]:
model_name = 'Atari_AE_TimeDist_Conv'

In [19]:
import os
#recording_dir='..'+sep_local+'..'+sep_local+'..'+sep_local+'recording'+sep_local + model_name
recording_dir=os.getcwd()+ sep_local  +'recording'+sep_local + model_name

In [20]:
from os.path import abspath
absolute = abspath(recording_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())


'..\\recording\\Atari_AE_tconv\\Atari_AE_tconv'

In [21]:
from training.autoencoding_basic.transformative.AE import autoencoder as AE

In [22]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':LATENT_DIM,
        'layers': enc_lays
    }
    ,
    
        {
        'name': 'generative', 
        'inputs_shape':LATENT_DIM,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [23]:
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(recording_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)

In [ ]:
ae = AE( 
    name=model_name,
    inputs_shape= inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=LATENT_DIM,
    variables_params=variables_params, 
    filepath=None
    )

In [26]:
ae.compile(metrics=None)

In [27]:
train_ds


TensorShape([36, 10])

In [ ]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver


In [ ]:
progbar = NotebookPrograssBar(leave_outer=False)

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [ ]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [ ]:
csv_dir = os.path.join(recording_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_gen_dir = os.path.join(recording_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

In [ ]:
sg = SampleGeneration(latent_shape=LATENT_DIM, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [ ]:
ae.fit(
    x=train_ds,
    steps_per_epoch=900,
    epochs=100, 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)